In [ ]:
# Research環境用関数
from quantopian.research import returns, symbols

# 時間範囲を指定
period_start = '2018-01-03'
period_end = '2018-01-04'

# 上記の時間範囲で、AAPLのリターンデータを照会する
aapl_returns = returns(
    assets=symbols('INTC'),
    start=period_start,
    end=period_end,
)

# 最初の10行のみ表示
aapl_returns.head(10)

In [ ]:
sym = "MU"
prices = get_pricing([sym], start_date="2018-1-1", end_date="2018-1-8", frequency='minute')

items = prices.transpose(2,1,0).items

prices.transpose(2,1,0)[items[0]].between_time("21:00",  "14:31")


In [ ]:
# Pipeline imports
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import Returns
from quantopian.pipeline.data.psychsignal import stocktwits

# Pipeline 定義
def make_pipeline():

    returns = Returns(window_length=2)
    sentiment = stocktwits.bull_minus_bear.latest
    msg_volume = stocktwits.total_scanned_messages.latest

    return Pipeline(
        columns={
            'daily_returns': returns,
            'sentiment': sentiment,
            'msg_volume': msg_volume,
        },
    )

# Pipeline 実行
data_output = run_pipeline(
    make_pipeline(),
    start_date=period_start,
    end_date=period_end
)


# AAPLのデータだけを取得
aapl_output = data_output.xs(
    symbols('AAPL'),
    level=1
)

# 描画
aapl_output.plot(subplots=True);

In [ ]:
# Pipeline と データセットをインポート
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.research import run_pipeline

# 移動平均を計算する関数をインポート
from quantopian.pipeline.factors import SimpleMovingAverage


def make_pipeline():
    # 日々の最終価格を取得
    close_price = USEquityPricing.close.latest

    # bull_minus_bearスコアの3日移動平均を演算
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )

    # pipelineに、最終価格と、センチメントスコアを入れて、返す
    return Pipeline(
        columns={
            'close_price': close_price,
            'sentiment_score': sentiment_score,
        }
    )

period_start = '2014-01-01'
period_end = '2014-12-31'

run_pipeline(make_pipeline(),
             period_start,
             period_end,)
             

In [ ]:
# Pipeline インポート
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS

# Pipeline 定義
def  make_pipeline():

    base_universe = QTradableStocksUS()

    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )

    # センチメントスコアに基づいて上位下位350銘柄のみを取得するフィルターを作成
    top_bottom_scores = (
        sentiment_score.top(350) | sentiment_score.bottom(350)
    )

    return Pipeline(
        columns={
            'sentiment_score': sentiment_score,
        },
        # 定義したフィルターとトレーディングユニバースのどちらにも入っている銘柄のみにスクリーニングする
        screen=(
            base_universe
            & top_bottom_scores
        )
    )

# run_pipeline インポート
from quantopian.research import run_pipeline

# 評価する期間を指定
period_start = '2013-01-01'
period_end = '2016-01-01'

# 指定期間で pipeline 実行
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date=period_start,
    end_date=period_end
)

In [ ]:
pipeline_output